In [208]:
import numpy as np
import os
import json
import pandas as pd
import glob
from tqdm import tqdm

In [201]:
# 取得某個階段的所有檔名
def get_filename(root_dir):
    tmp = glob.glob(root_dir + '/*.json')
        
    return tmp

In [219]:
def Station2CSV(p):
    sta = pd.DataFrame()
    
    for k in p.keys():
        try:
            # 看測站內有多少組波形資料
            n_data = p[k]['numberOfData']
            
            for w in range(n_data):
                tmp = {}
                 
                tmp['event'] = str(p['event']) # PK
                tmp['distance'] = p[k]['distance']
                tmp['p_weight'] = p[k]['p_weight']
                tmp['s_weight'] = p[k]['s_weight']
                tmp['network'] = p[k][str(w)]['network']
                tmp['location'] = p[k][str(w)]['location']
                tmp['sampling_rate'] = p[k][str(w)]['sampling_rate']
                tmp['instrument'] = p[k][str(w)]['instrument']
                tmp['starttime'] = p[k][str(w)]['starttime'][:-1]
                tmp['p_arrival_time'] = p[k][str(w)]['p_arrival_time']
                tmp['s_arrival_time'] = p[k][str(w)]['s_arrival_time']
                tmp['pga'] = p[k][str(w)]['pga']
                tmp['pgv'] = p[k][str(w)]['pgv']
                tmp['intensity'] = p[k][str(w)]['intensity']
                tmp['waveFile'] = p[k][str(w)]['waveFile']
                tmp['instrument_available'] = p[k][str(w)]['DataAvailable']['instrument']
                tmp['intensity_available'] = p[k][str(w)]['DataAvailable']['intensity']
                tmp['pga_available'] = p[k][str(w)]['DataAvailable']['pga']
                tmp['pgv_available'] = p[k][str(w)]['DataAvailable']['pgv']
                tmp['Stime_available'] = p[k][str(w)]['DataAvailable']['Stime']
               
                idx = k+'_'+str(w)
                info = pd.DataFrame(tmp, index=[idx])
                sta = pd.concat([sta, info])
           
        except Exception as e:
            #print(e)
            pass
        
    return sta

In [220]:
year = '2012'
root_path = '/mnt/nas6/CWBSN/' + year
save_path = '/mnt/nas6/CWBSN/' + year + '/csv'
#root_path = '/mnt/nas6/CWBSN/example'
#save_path = "/mnt/nas6/CWBSN/example/csv/"
files = get_filename(root_path)

In [221]:
if not os.path.exists(save_path):
    print('creating directiory: %s' %(save_path))
    os.mkdir(save_path)

In [222]:
event_df = pd.DataFrame()
sta_df = pd.DataFrame()

for file in tqdm(files):
    f = open(os.path.join(root_path, file), 'r')
    p = json.load(f)
    df = pd.json_normalize(p)
    
    event_df = pd.concat([event_df, pd.concat([df.loc[:, :'lon'], df['quality'], df['event']], axis=1)])
    sta_df = pd.concat([sta_df, Station2CSV(p)])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1253/1253 [05:19<00:00,  3.92it/s]


In [223]:
event_df.to_csv(os.path.join(save_path, 'event.csv'), index=False)
sta_df.to_csv(os.path.join(save_path, 'station.csv'))